# Battle of the Neighborhoods
Introduction
This project is for who is planning to open a Coffee House in Ankara, Turkey. This project suggests the best locations for Coffee Houses in Ankara. Ankara is the capital of Turkey with a population of 5M. Turkish culture has deep roots in coffee. Also, it is getting popular because of its affordable prices. Coffee consumption has increased 13% and average consumption is 1.1 kg/person in 2018. This report explores which neighborhoods of Ankara have the most as well as the best Coffee Houses. Also, this project answers the questions “Where should I open an Coffee House?” and “Where should I stay If I want a tasty coffee?”

###### Data
District of Ankara are obtained from https://en.wikipedia.org/wiki/Ankara_Province
Latitude and Longitude values are obtained by using "geocoder".
All data related to locations will be obtaine by using FourSquare API and Python Libraries.

In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
#!conda install -c conda-forge geopy --yes
import geocoder

In [123]:
wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wiki_page = requests.get(wiki_link, headers = headers)
wiki_page

<Response [200]>

In [124]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [125]:

rows = table.find_all('tr')


In [126]:
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [127]:
df_ankara = pd.DataFrame(columns = columns)

In [128]:

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [129]:
df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [130]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [131]:

districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [132]:
df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [133]:

df_ankara_loc.drop(columns="Population (2017)", axis=1, inplace=True)

In [134]:

df_ankara_loc.drop(columns="Density (per km²)", axis=1, inplace=True)

In [135]:

df_ankara_loc.drop(columns="Area (km²)", axis=1, inplace=True)

In [136]:
df_ankara_loc.head()

,District,Latitude,Longitude
0,Akyurt,40.13082,33.08719
1,Altındağ,39.94171,32.85445
2,Ayaş,40.01516,32.33270
3,Bala,39.55391,33.12352
4,Beypazarı,40.16811,31.92052


In [137]:
import numpy as np
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print("Libraries imported")

Libraries imported


In [138]:
from geopy.geocoders import Nominatim 

address = "Çankaya, Ankara"

geolocator = Nominatim(user_agent = "Ankara_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Ankara are {}, {}.".format(latitude, longitude))

The geographical coordinates of Ankara are 39.9207893, 32.8540412.


In [139]:
map_ankara = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_ankara_loc["Latitude"], df_ankara_loc["Longitude"], df_ankara_loc["District"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_ankara)  
    
map_ankara

In [140]:
CLIENT_ID = "KQGAMAISIPOXFVVGVUMKMYLAV2342L3AU1RJZNAEBZSLE0WN"
CLIENT_SECRET = "SSUIEW0FOMOOM43JYLSHYODVBW5IVF1KJJDVMWOZ401M1PJD" 
VERSION = "20210728"

In [141]:
df_ankara_loc.loc [0, "District"]
df_ankara_loc.loc [0, "District"]

'Akyurt'

In [142]:
neighborhood_latitude = df_ankara_loc.loc[0, "Latitude"]
neighborhood_longitude = df_ankara_loc.loc[0, "Longitude"] 

neighborhood_name = df_ankara_loc.loc[0, "District"] 

print("Latitude and longitude values of the neighborhood {} are {}, {}.".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of the neighborhood Akyurt are 40.13082000000003, 33.08719000000008.


In [143]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KQGAMAISIPOXFVVGVUMKMYLAV2342L3AU1RJZNAEBZSLE0WN&client_secret=SSUIEW0FOMOOM43JYLSHYODVBW5IVF1KJJDVMWOZ401M1PJD&v=20210728&ll=40.13082000000003,33.08719000000008&radius=500&limit=100'

In [144]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6100e7c27304d162674b0d22'},
 'response': {'headerLocation': 'Akyurt',
  'headerFullLocation': 'Akyurt, Ankara',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 40.13532000450003,
    'lng': 33.093064640939495},
   'sw': {'lat': 40.126319995500026, 'lng': 33.08131535906066}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5049c781e4b0a7a32d840eb1',
       'name': 'Meşhur Köfteci Bodur',
       'location': {'address': 'Cumhuriyet Meydani',
        'crossStreet': 'Cumhuriyet Meydani',
        'lat': 40.13071902888477,
        'lng': 33.08677622815274,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.13071902888477,
          'lng': 33.08677622815274}],
        '

In [145]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [146]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,Meşhur Köfteci Bodur,Turkish Restaurant,40.130719,33.086776
1,Akyurt kültür parkı,Park,40.133123,33.085071
2,Yurtalan Restaurant,Kebab Restaurant,40.132672,33.084306
3,Onur pide ve kebap salonu,Kebab Restaurant,40.130807,33.086422
4,Beyazıt Sofrası,Steakhouse,40.130542,33.081901
5,Yurtalan,Turkish Restaurant,40.132743,33.084385
6,Yurtalan Restaurant,Doner Restaurant,40.132641,33.084492
7,Benliler Supermarket,Convenience Store,40.130252,33.081423
8,şişman pasta&cafe,Café,40.130820,33.085025
9,Akyurt Sofrası,Turkish Restaurant,40.130854,33.086709


In [147]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


In [148]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [149]:

ankara_venues = getNearbyVenues(names=df_ankara_loc['District'],
                                   latitudes=df_ankara_loc['Latitude'],
                                   longitudes=df_ankara_loc['Longitude']
                                  )

Akyurt
Altındağ
Ayaş
Bala
Beypazarı
Çamlıdere
Çankaya
Çubuk
Elmadağ
Etimesgut
Evren
Gölbaşı
Güdül
Haymana
Kahramankazan
Kalecik
Keçiören
Kızılcahamam
Mamak
Nallıhan
Polatlı
Pursaklar
Sincan
Şereflikoçhisar
Yenimahalle
Urban (9 districts)Altındağ, Çankaya, Etimesgut, Gölbaşı, Keçiören, Mamak, Pursaklar, Sincan, Yenimahalle
TOTAL


In [150]:
print(ankara_venues.shape)
ankara_venues.head(250)

(1664, 7)


,District,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.13082,33.08719,Meşhur Köfteci Bodur,40.130719,33.086776,Turkish Restaurant
1,Akyurt,40.13082,33.08719,Akyurt kültür parkı,40.133123,33.085071,Park
2,Akyurt,40.13082,33.08719,Yurtalan Restaurant,40.132672,33.084306,Kebab Restaurant
3,Akyurt,40.13082,33.08719,Beyazıt Sofrası,40.130542,33.081901,Steakhouse
4,Akyurt,40.13082,33.08719,Onur pide ve kebap salonu,40.130807,33.086422,Kebab Restaurant
...,...,...,...,...,...,...,...
245,Beypazarı,40.16811,31.92052,Seçkin Çil Parkı,40.158702,31.926827,Park
246,Beypazarı,40.16811,31.92052,Sağlık Meslek Spor Salonu,40.173256,31.933203,Trail
247,Beypazarı,40.16811,31.92052,Tolunay Özaka Sağlık Meslek Lisesi,40.173212,31.933535,Volleyball Court
248,Beypazarı,40.16811,31.92052,optimum,40.157738,31.913723,Shopping Mall


In [151]:

ankara_venues.groupby("District").count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Akyurt,31,31,31,31,31,31
Altındağ,100,100,100,100,100,100
Ayaş,36,36,36,36,36,36
Bala,14,14,14,14,14,14
Beypazarı,70,70,70,70,70,70
Elmadağ,52,52,52,52,52,52
Etimesgut,99,99,99,99,99,99
Evren,5,5,5,5,5,5
Gölbaşı,100,100,100,100,100,100


In [152]:
print('There are {} unique venue categories.'.format(len(ankara_venues['Venue Category'].unique())))

There are 232 unique venue categories.


In [153]:
ankara_onehot = pd.get_dummies(ankara_venues[['Venue Category']], prefix="", prefix_sep="")

ankara_onehot['Neighborhood'] = ankara_venues['District'] 

fixed_columns = [ankara_onehot.columns[-1]] + list(ankara_onehot.columns[:-1])
ankara_onehot = ankara_onehot[fixed_columns]

ankara_onehot

,Neighborhood,ATM,Advertising Agency,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Watch Shop,Water Park,Waterfront,Wedding Hall,Women's Store,Yoga Studio
0,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1660,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1661,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1662,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
ankara_onehot.shape

(1664, 233)

In [155]:
ankara_grouped = ankara_onehot.groupby('Neighborhood').mean().reset_index()
ankara_grouped

,Neighborhood,ATM,Advertising Agency,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Watch Shop,Water Park,Waterfront,Wedding Hall,Women's Store,Yoga Studio
0,Akyurt,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.032258,0.000000
1,Altındağ,0.000000,0.00,0.000000,0.03,0.000000,0.000000,0.02,0.020000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
2,Ayaş,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
3,Bala,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
4,Beypazarı,0.000000,0.00,0.000000,0.00,0.000000,0.014286,0.00,0.000000,0.014286,...,0.00,0.00000,0.000000,0.014286,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
5,Elmadağ,0.019231,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.019231,0.0,0.00000,0.000000,0.000000
6,Etimesgut,0.000000,0.00,0.000000,0.00,0.000000,0.040404,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.010101
7,Evren,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.2,0.00000,0.000000,0.000000
8,Gölbaşı,0.000000,0.00,0.000000,0.00,0.000000,0.040000,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.010000,0.0,0.00000,0.000000,0.000000
9,Güdül,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000


In [156]:
ankara_grouped.shape

(27, 233)

In [157]:
num_top_venues = 10

for hood in ankara_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ankara_grouped[ankara_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akyurt----
                venue  freq
0                Farm  0.13
1                Café  0.13
2  Turkish Restaurant  0.10
3    Kebab Restaurant  0.06
4                Park  0.06
5    Botanical Garden  0.03
6  Basketball Stadium  0.03
7       Big Box Store  0.03
8            Mountain  0.03
9                 Gym  0.03


----Altındağ----
                    venue  freq
0                    Café  0.11
1          History Museum  0.07
2           Historic Site  0.05
3                   Hotel  0.05
4      Turkish Restaurant  0.04
5           Jewelry Store  0.04
6                 Theater  0.04
7              Restaurant  0.04
8             Coffee Shop  0.03
9  Furniture / Home Store  0.03


----Ayaş----
                     venue  freq
0       Turkish Restaurant  0.14
1               Restaurant  0.08
2                     Café  0.08
3                     Lake  0.06
4         Kebab Restaurant  0.03
5  Fruit & Vegetable Store  0.03
6            Grocery Store  0.03
7                     Pool 

In [158]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [176]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
ankara_venues_sorted = pd.DataFrame(columns=columns)
ankara_venues_sorted['Neighborhood'] = ankara_grouped['Neighborhood']

for ind in np.arange(ankara_grouped.shape[0]):
    ankara_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ankara_grouped.iloc[ind, :], num_top_venues)

ankara_venues_sorted.head(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Café,Farm,Turkish Restaurant,Kebab Restaurant,Park,Botanical Garden,Big Box Store,Steakhouse,Basketball Stadium,Basketball Court
1,Altındağ,Café,History Museum,Hotel,Historic Site,Jewelry Store,Turkish Restaurant,Theater,Restaurant,Steakhouse,Antique Shop


In [165]:
from sklearn.cluster import KMeans

print('Initialized')

Initialized


In [178]:
## Cluster Locality
## Run k-means to cluster the Locality into 5 clusters.

# set number of clusters
kclusters = 5

Ankara_clustering = ankara_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ankara_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
kmeans.labels_.shape


,ATM,Advertising Agency,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Watch Shop,Water Park,Waterfront,Wedding Hall,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.032258,0.0
1,0.0,0.0,0.0,0.03,0.0,0.000000,0.02,0.02,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.00,0.0,0.014286,0.00,0.00,0.014286,0.0,...,0.0,0.0,0.0,0.014286,0.0,0.0,0.0,0.0,0.000000,0.0


In [180]:
ankara_merged = df_ankara
ankara_merged.head(2)

ankara_merged = df_ankara
ankara_merged['Cluster Labels'] = kmeans.labels_

ankara_merged = ankara_merged.join(ankara_venues_sorted.set_index('Neighborhood'), on='District')
ankara_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Café,Farm,Turkish Restaurant,Kebab Restaurant,Park,Botanical Garden,Big Box Store,Steakhouse,Basketball Stadium,Basketball Court
1,Altındağ,Café,History Museum,Hotel,Historic Site,Jewelry Store,Turkish Restaurant,Theater,Restaurant,Steakhouse,Antique Shop
2,Ayaş,Turkish Restaurant,Restaurant,Café,Lake,Plaza,BBQ Joint,Food Court,Snack Place,Farmers Market,Fruit & Vegetable Store
3,Bala,Business Service,Electronics Store,Park,Bakery,Café,Mountain,Turkish Restaurant,Steakhouse,Big Box Store,Convenience Store
4,Beypazarı,Turkish Restaurant,History Museum,Motel,Bed & Breakfast,Bakery,Tea Room,Restaurant,Park,Scenic Lookout,Hotel
5,Elmadağ,Café,Park,Campground,Dessert Shop,Kebab Restaurant,Bar,Convenience Store,Pub,Cigkofte Place,Steakhouse
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Arcade,Park,Steakhouse,Dessert Shop,Bagel Shop,Shopping Mall,Kebab Restaurant
7,Evren,Grocery Store,Waterfront,Steakhouse,Park,River,Fast Food Restaurant,Event Space,Farm,Farmers Market,Fish Market
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Breakfast Spot,Coffee Shop,Arcade,Middle Eastern Restaurant,Bakery,Dessert Shop
9,Güdül,Botanical Garden,Pharmacy,Park,Castle,Turkish Restaurant,BBQ Joint,Moving Target,Department Store,Food & Drink Shop,Food


In [181]:
ks = 3

ankara_grouped_clustering = ankara_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=ks, random_state=0).fit(ankara_grouped_clustering)

kmeans.labels_[1:10]

array([1, 1, 0, 1, 1, 1, 0, 1, 2])

In [187]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ks)
ys = [i+x+(i*x)**2 for i in range(ks)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ankara_merged['Latitude'], ankara_merged['Longitude'], ankara_merged['District'], ankara_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

IndexError: list index out of range

In [183]:

ankara_merged.loc[ankara_merged['Cluster Labels'] == 0, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Ayaş,Turkish Restaurant,Restaurant,Café,Lake,Plaza,BBQ Joint,Food Court,Snack Place,Farmers Market,Fruit & Vegetable Store
10,Evren,Grocery Store,Waterfront,Steakhouse,Park,River,Fast Food Restaurant,Event Space,Farm,Farmers Market,Fish Market
14,Kahramankazan,Park,Café,Restaurant,Grocery Store,Mobile Phone Shop,Turkish Restaurant,Ice Cream Shop,Soccer Stadium,Burmese Restaurant,Scenic Lookout
16,Keçiören,Café,Bakery,Turkish Restaurant,Coffee Shop,Ice Cream Shop,Cosmetics Shop,Park,Restaurant,Gym,Music Venue
26,TOTAL,Soup Place,Park,Bakery,Coffee Shop,Rest Area,Steakhouse,Bistro,Soccer Stadium,Food Court,Middle Eastern Restaurant


In [184]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 1, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Çubuk,Turkish Restaurant,Café,Bistro,Gym / Fitness Center,Park,Shopping Mall,Mobile Phone Shop,Pharmacy,Convenience Store,Kebab Restaurant


In [185]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 2, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Café,Farm,Turkish Restaurant,Kebab Restaurant,Park,Botanical Garden,Big Box Store,Steakhouse,Basketball Stadium,Basketball Court
1,Altındağ,Café,History Museum,Hotel,Historic Site,Jewelry Store,Turkish Restaurant,Theater,Restaurant,Steakhouse,Antique Shop
3,Bala,Business Service,Electronics Store,Park,Bakery,Café,Mountain,Turkish Restaurant,Steakhouse,Big Box Store,Convenience Store
4,Beypazarı,Turkish Restaurant,History Museum,Motel,Bed & Breakfast,Bakery,Tea Room,Restaurant,Park,Scenic Lookout,Hotel
5,Çamlıdere,Forest,Art Museum,Plaza,Convenience Store,BBQ Joint,Park,Turkish Restaurant,Yoga Studio,Fish & Chips Shop,Farmers Market
6,Çankaya,Café,Bookstore,Bar,Turkish Restaurant,Cosmetics Shop,Coffee Shop,Pub,Sandwich Place,Dessert Shop,Optical Shop
8,Elmadağ,Café,Park,Campground,Dessert Shop,Kebab Restaurant,Bar,Convenience Store,Pub,Cigkofte Place,Steakhouse
11,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Breakfast Spot,Coffee Shop,Arcade,Middle Eastern Restaurant,Bakery,Dessert Shop
12,Güdül,Botanical Garden,Pharmacy,Park,Castle,Turkish Restaurant,BBQ Joint,Moving Target,Department Store,Food & Drink Shop,Food
13,Haymana,Hotel,Restaurant,Hot Spring,Water Park,Diner,Café,Bathing Area,Tea Room,Bath House,Supermarket


In [186]:

ankara_merged.loc[ankara_merged['Cluster Labels'] == 3, ankara_merged.columns[[0] + list(range(5, ankara_merged.shape[1]))]]

,District,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Etimesgut,Café,Restaurant,Arcade,Park,Steakhouse,Dessert Shop,Bagel Shop,Shopping Mall,Kebab Restaurant
